In [ ]:
# ******************************************
# Ionel Rodriguez 8-764-368
# Modelos Predictivos
# ******************************************

import pandas
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import warnings
import sys

sys.path.append('./.')
import comun

warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = "monospace"

In [ ]:
# Se cargan los datos desde la base de datos. Se limpia y ajusta la base de datos para los análisis siguientes.
# Se utilizan funciones helpers que se encuentran en el módulo "comun" ubicado en la misma carpeta donde se encuentra este notebook.

df_main = comun.load_base_dataframe()
df_numeric = df_main[['fecha', 'marca_id', 'modelo_id', 'cantidad', 'costo']]

In [ ]:
# Se inspecciona las primeras 15 filas para ver como quedó la base de datos
df_main.head(15)

In [ ]:
# Mostrar los tipos de datos de cada columna
df_main.info()

In [ ]:
# Detalles estadísticos básicos de las columnas
# Mostrar más info que df.describe()

pandas.DataFrame([
		df_numeric.count(),
		df_numeric.sum(numeric_only=True),
		df_numeric.mean(),
		df_numeric.median(),
		df_numeric.mode().iloc[0],
		df_numeric.min(),
		df_numeric.max(),
		df_numeric.std(numeric_only=True),
		df_numeric.sem(numeric_only=True),
		df_numeric.var(numeric_only=True),
		df_numeric.skew(numeric_only=True),
		df_numeric.kurt(numeric_only=True),
		df_numeric.quantile(0.25),
		df_numeric.quantile(0.75),
	],
	index=['count', 'sum', 'mean', 'median', 'mode', 'min', 'max', 'std dev.', 'std error', 'variance', 'skewness', 'kurtosis', 'quantile 25%', 'quantile 75%']
)


In [ ]:
# Se genera una matriz de correlación de las variables numéricas

matrix = df_numeric.corr()

plt.figure(figsize=(8,6))
sns.heatmap(matrix, linewidth=0.5,cmap='RdYlGn', vmax=1, vmin=-1, annot=matrix.to_numpy(), annot_kws={'size':8})
plt.title('Matriz de Correlación', fontsize=12)
plt.show()


In [ ]:
# Se generan gráficos que comparan la cantidad de los modelos vs los costos de los mismos modelos de auto.

def build_grouped_bars(grouping_var, legend_align='upper left'):
	df_sorted = df_main.filter([grouping_var, 'cantidad', 'costo'], axis=1).groupby(grouping_var, sort=False, as_index=False).sum().sort_values('cantidad', ascending=False)
	df_sorted['costo'] /= 1000


	names = df_sorted[grouping_var]
	x = np.arange(len(names))
	fig, ax = plt.subplots(layout='constrained')
	w = 0.4
	mult = 0

	for var, items in df_sorted.items():
		if var != grouping_var:
			rects = ax.bar(x +  w*mult, items, w, label=var)
			ax.bar_label(rects, padding=1)
			mult += 1
	
	ax.set_title('{} vs Cantidad vs Costo (en miles)'.format(grouping_var.capitalize()))
	ax.set_xticks(x + w/2, names)
	ax.legend(loc=legend_align, ncols=1)

	plt.show()

build_grouped_bars('modelo')

In [ ]:
# Similar a la gráfica anterior, solo que ahora se evalúan las marcas de autos.

build_grouped_bars('marca', legend_align='upper right')

In [ ]:
# Se generan boxplots de los modelos de auto para mostrar sus respectivos outliers.

plt.figure().set_figwidth(8)
sns.boxplot(y=df_main.modelo, x=df_main.cantidad)

In [ ]:
# Se efectúa una limpieza de los datos para remover los outliers y se vuelve a generar los boxplots para comprobar.

df_no_outlier = comun.remove_outliers(df_main, 'cantidad')

plt.figure().set_figwidth(8)
sns.boxplot(y=df_no_outlier.modelo, x=df_no_outlier.cantidad)
#sns.swarmplot(x='cantidad', y='modelo', data=df_no_outlier, color='black', alpha=0.4)
plt.show()
